## Fit Strong Lens model on ECDFS_G15422 with Lenstronomy

- author : Sylvie Dagoret-Campagne
- creation date : 2025-06-16
- update : 2025-06-18
- last update : 2025-06-19 : Search where are located the image of the source and now the fit works well
- last update : 2025-06-24

https://stackoverflow.com/questions/59699193/how-to-obtain-2d-cutout-of-an-image-from-a-skycoord-position

In [ ]:
# For Angle conversion
import matplotlib.pyplot as plt
from astropy.coordinates import Angle
import astropy.units as u
from astropy.coordinates import SkyCoord
import pandas as pd
import numpy as np
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
from astropy.visualization import simple_norm, ZScaleInterval,PercentileInterval

In [ ]:
from IPython.display import Image, display

In [ ]:
plt.rcParams["figure.figsize"] = (12,8)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams['legend.fontsize']=  12
plt.rcParams['font.size'] = 12

## Configuration

### Target in ECDFS

- path to article : https://arxiv.org/pdf/1104.0931
- visual selection of the tiles : https://archive.stsci.edu/prepds/gems/browser.html
- path file download : https://archive.stsci.edu/pub/hlsp/gems/v_mk1/

In [ ]:
fig_table1 = "input_figs/table1_gemcandidates.png"
fig_table2 = "input_figs/table2_gemcandidates.png"

In [ ]:
Image(url= fig_table1,width=800)

In [ ]:
Image(url= fig_table2,width=500)

In [ ]:
#15422 44 03:32:38.21 –27:56:53.2 
ra1 = "03:32:38.21 hours"
dec1 = "-27:56:53.2 degrees"
tile1 = 44

#34244 94 03:32:06.45 –27:47:28.6 
ra2 = "03:32:06.45 hours"
dec2 = "-27:47:28.6 degrees"
tile2 = 94
# Je ne trouve pas cette tile ==> FindTileForCutoutGEM
tile2 = 32

#40173 35 03:33:19.45 –27:44:50.0 
ra3 = "03:33:19.45 hours"
dec3 = "-27:44:50.0 degrees"
tile3 = 35

#43242 45 03:31:55.35 –27:43:23.5 
ra4 = "03:31:55.35 hours"
dec4 = "-27:43:23.5 degrees"
tile4 = 45

#46446 47 03:31:35.94 –27:41:48.2 
ra5 = "03:31:35.94 hours"
dec5 = "-27:41:48.2 degrees"
tile5 = 47

#12589 03:31:24.89 −27:58:07.0
ra6 = "03:31:24.89 hours"
dec6 = "-27:58:07.0 degrees"
tile6 = 17

#43797 03:31:31.74 −27:43:00.8 
ra7 = "03:31:31.74 hours"
dec7 = "-27:43:00.8 degrees"
tile7 = 47

#28294 03:31:50.54 −27:50:28.4 
ra8 = "03:31:50.54 hours"
dec8 = "-27:50:28.4 degrees"
tile8 = 33

#36857 03:31:53.24 −27:46:18.9
ra9 = "03:31:53.24 hours"
dec9 = "-27:46:18.9 degrees"
tile9 = 38

#36714 03:32:59.78 −27:46:26.4 
ra10 = "03:32:59.78 hours"
dec10 = "-27:46:26.4 degrees"
tile10 = 37


In [ ]:
lsstcomcam_targets = {}
# high rank
lsstcomcam_targets["ECDFS_G15422"] = {"field_name": "GEMS-15422", "ra": 53.159208333333325, "dec": -27.94811111111111,"tile":tile1}
lsstcomcam_targets["ECDFS_G34244"] = {"field_name": "GEMS-34244", "ra": 53.02687499999999 , "dec": -27.79127777777778,"tile":tile2}
lsstcomcam_targets["ECDFS_G40173"] = {"field_name": "GEMS-40173", "ra": 53.33104166666666 , "dec": -27.747222222222224,"tile":tile3}
lsstcomcam_targets["ECDFS_G43242"] = {"field_name": "GEMS-43242", "ra": 52.980624999999996 , "dec": -27.72319444444444,"tile":tile4}
lsstcomcam_targets["ECDFS_G46446"] = {"field_name": "GEMS-46446", "ra": 52.89975 , "dec": -27.696722222222224,"tile":tile5}

# low rank
lsstcomcam_targets["ECDFS_G12589"] = {"field_name": "GEMS-12589", "ra": 52.85370833333333, "dec": -27.96861111111111,"tile":tile6}
lsstcomcam_targets["ECDFS_G43797"] = {"field_name": "GEMS-43797", "ra": 52.88224999999999, "dec": -27.71688888888889,"tile":tile7}

lsstcomcam_targets["ECDFS_G28294"] = {"field_name": "GEMS-28294", "ra": 52.960583333333325 , "dec": -27.84122222222222,"tile":tile8}
lsstcomcam_targets["ECDFS_G6857"] = {"field_name": "GEMS-6857", "ra": 52.97183333333333 , "dec": -27.771916666666666,"tile":tile9}
lsstcomcam_targets["ECDFS_G36714"] = {"field_name": "GEMS-36714", "ra": 53.249083333333324, "dec": -27.773999999999997,"tile":tile10}


In [ ]:
df = pd.DataFrame(lsstcomcam_targets).T

In [ ]:
# candidates
key = "ECDFS_G15422"
#key = "ECDFS_G34244"
#key = "ECDFS_G40173"
#key= "ECDFS_G43242"
#key= "ECDFS_G46446"

# unknown
#key = "ECDFS_G12589"
#key = "ECDFS_G43797"
#key = "ECDFS_G28294"
#key = "ECDFS_G6857"
#key = "ECDFS_G36714"

the_target = lsstcomcam_targets[key]
target_ra = the_target["ra"]
target_dec = the_target["dec"]
target_name = the_target["field_name"]
tile_num = the_target["tile"]
coord = SkyCoord(ra=target_ra, dec=target_dec, unit=(u.deg, u.deg))

target_title = (
    the_target["field_name"] + f" (ra,dec) = ({target_ra:.2f},{target_dec:.2f}) "
)

In [ ]:
tile_num

### Access to remote files

- visual selection of the tiles : https://archive.stsci.edu/prepds/gems/browser.html
- path file download : https://archive.stsci.edu/pub/hlsp/gems/v_mk1/

### Config

- http://archive.stsci.edu/pub/hlsp/gems/v_mk1/h_gems_v35_mk1.fits
- url = f"http://archive.stsci.edu/pub/hlsp/gems/z_mk1/h_gems_z{tile_num}_mk1.fits"

In [ ]:
# select your bands
SELECT_VBAND = "v"
SELECT_ZBAND = "z"
SELECT_BAND = SELECT_ZBAND

#### define the cutoutfilename path

In [ ]:
cutout_inputpath = f"cutout_gems_{key}_b{SELECT_BAND}.fits"
cutout_inputpath_wt = f"cutout_gems_{key}_b{SELECT_BAND}_wt.fits"

### Load image and weight


🌈 Liste des stretch disponibles dans simple_norm
Tu peux utiliser ces chaînes de caractères comme valeur de stretch :
Valeur stretch	Description
- "linear"	Échelle linéaire simple (valeurs brutes entre min_cut et max_cut)
- "sqrt"	Échelle racine carrée – met en valeur les faibles intensités
- "power"	Échelle puissance (x^power)
- "log"	Échelle logarithmique – bon pour étendre la dynamique
- "asinh"	Échelle arcsinus hyperbolique – lisse, bien adaptée aux images astronomiques
- "sinh"	Hyperbolique sinus – moins courant
- "histeq"	Égalisation d'histogramme – utile pour répartir les contrastes

In [ ]:
# Calcul des coupures automatiques avec ZScale
#interval = ZScaleInterval()
#interval = PercentileInterval(98) 
#vmin_v, vmax_v = interval.get_limits(data_v)

# Normalisation avec stretch asinh
#norm_v = simple_norm(data_v, stretch='asinh', min_cut=vmin_v, max_cut=vmax_v)
#norm_v = simple_norm(data_v, 'asinh')

In [ ]:
def load_fits_image_with_norm(path):
    with fits.open(path) as hdul:
        data = hdul[0].data
        wcs = WCS(hdul[0].header)
        header = hdul[0].header
    vmin, vmax = ZScaleInterval().get_limits(data)
    norm = simple_norm(data, stretch='asinh', vmin=vmin, vmax=vmax)
    return data, wcs, norm, header

In [ ]:
def add_EastNorthFrame(x0,y0,ax,width =30, dwidth = 5):
    """
    """
    #x0, y0 = 50, 150  # Coordonnée de base pour les flèches
    # Ajout de la flèche vers le Nord
    ax.annotate('', xy=(x0, y0 + width), xytext=(x0, y0),
             arrowprops=dict(facecolor='white', width=3, headwidth=10))
    ax.text(x0, y0 + (width+dwidth), 'N', color='white', ha='center', va='bottom', fontsize=14,fontweight="bold")

    # Ajout de la flèche vers l'Est
    ax.annotate('', xy=(x0 - width, y0), xytext=(x0, y0),
             arrowprops=dict(facecolor='white', width=3, headwidth=10))
    ax.text(x0 - (width+dwidth), y0, 'E', color='white', ha='left', va='center', fontsize=14,fontweight="bold")
    

In [ ]:
# Charger les données

data, wcs, norm,header = load_fits_image_with_norm(cutout_inputpath)
data_wt, wcs_wt, norm_wt,header_wt = load_fits_image_with_norm(cutout_inputpath_wt)


fig = plt.figure(figsize=(14, 7))


ax1 = fig.add_subplot(1, 2, 1, projection=wcs)
im1 = ax1.imshow(data, origin='lower', cmap='gray', norm=norm)
ax1.set_title(f'Flux {key} {SELECT_BAND} band')
ax1.set_xlabel('RA')
ax1.set_ylabel('Dec')
ax1.grid(color="blue",linestyle=":")
add_EastNorthFrame(50,150,ax1)



ax2 = fig.add_subplot(1, 2, 2, projection=wcs_wt)
im2 = ax2.imshow(data_wt, origin='lower', cmap='gray', norm=norm_wt)
ax2.set_title(f'Weight {key} {SELECT_BAND}  band')
ax2.set_xlabel('RA')
ax2.set_ylabel('')
ax2.grid(color="blue",linestyle=":")
add_EastNorthFrame(50,150,ax2)

# Mise en page
plt.tight_layout()
plt.show()


## Study peaks 

The goal is to find where are located the possible images of the sources in the image

#### Work on WCS

In [ ]:
from astropy.wcs.utils import proj_plane_pixel_scales

In [ ]:
# Donne les échelles angulaires en degrés/pixel pour chaque axe
pixscale_deg = proj_plane_pixel_scales(wcs)  # [deg/pixel] en X et Y
pixscale_arcsec = pixscale_deg * 3600        # [arcsec/pixel]
print(f"Pixel scale : {pixscale_arcsec[0]:.5f} x {pixscale_arcsec[1]:.5f} arcsec/pixel")
# pixel scale in arcsec
pixel_scale = np.average(pixscale_arcsec)

🧠 Détails techniques :
La matrice CD (coordinate description) est une matrice 2×2 qui donne la transformation linéaire (rotation, échelle, cisaillement) entre les pixels et les coordonnées en degrés (RA/Dec, etc).
Si l’en-tête FITS contient seulement PCi_j et CDELTi, la matrice CD est implicitement obtenue par :
$ CD = CDELT \times  PC $
Astropy fournit la méthode .get_cd() pour obtenir la matrice CD dans tous les cas.

In [ ]:
cd = wcs.wcs.cd  # si présent
if cd is None:
    # Sinon, recompose à partir de PC et CDELT
    cd = wcs.wcs.get_cd()

In [ ]:
print("Matrice CD (de pixels vers angles en degrés):")
print(cd)

In [ ]:
transform_pix2angle = cd*3600.
print("Matrice CD (de pixels vers angles en arcsec):")
print(transform_pix2angle)

In [ ]:
# Convertit le pixel (0, 0) en coordonnées du ciel (RA, Dec)
ra_at_xy_0, dec_at_xy_0 = wcs.wcs_pix2world(0, 0, 0)  # le 3e argument = 0 → origine '0-based' comme Python

In [ ]:
delta_ra_at_xy_0 = (ra_at_xy_0 - target_ra)*3600
delta_dec_at_xy_0 = (dec_at_xy_0 - target_dec)*3600
print(f"relative position of corner x=0 y=0 is delta_ra = {delta_ra_at_xy_0:.3f} arcsec, delta_dec = {delta_dec_at_xy_0:.3f} arcsec ")

### Image smoothing

In [ ]:
fwhm_arcsec = 0.11 # arcsec
fwhm_pixel = fwhm_arcsec/pixel_scale
sigma_pixel_smoothing = fwhm_pixel/2.36
sigma_pixel_smoothing = int(np.ceil(sigma_pixel_smoothing))
print(f"sigma_pixel_smoothing = {sigma_pixel_smoothing}")

In [ ]:
from scipy.ndimage import gaussian_filter
smoothed_image = gaussian_filter(data, sigma= sigma_pixel_smoothing )
print(f"smoothed image shape : {smoothed_image.shape}")

In [ ]:
from skimage.feature import peak_local_max

# Trouver les maxima locaux isoles (min_distance = 10) sur l'image lissée 
coordinates = peak_local_max(smoothed_image, min_distance=10)

In [ ]:
# Extraire les valeurs correspondantes dans l'image lissée
values = smoothed_image[coordinates[:, 0], coordinates[:, 1]]

In [ ]:
# Créer un tableau combiné (x, y, valeur)
# Réorganiser en (x, y, value)
x = coordinates[:, 1]
y = coordinates[:, 0]
results = np.column_stack((x, y, values))  # shape: (N, 3)

In [ ]:
df_maxima = pd.DataFrame(results, columns=["x", "y", "max"])
df_maxima = df_maxima.sort_values(by="max", ascending=False)

In [ ]:
fig = plt.figure(figsize=(16, 16))

NCONT = 50

ax1 = fig.add_subplot(2, 2, 1, projection=wcs)
im1 = ax1.imshow(data, origin='lower', cmap='jet', norm=norm)
ax1.set_title(f'Flux {key} {SELECT_BAND} band')
ax1.set_xlabel('RA')
ax1.set_ylabel('Dec')
ax1.grid(color="blue",linestyle=":")
ax1.contour(data, levels=NCONT, colors='white', linewidths=1)
add_EastNorthFrame(40,155,ax1)



ax2 = fig.add_subplot(2, 2, 2, projection=wcs)
im2 = ax2.imshow(smoothed_image, origin='lower', cmap='jet', norm=norm)
ax2.set_title(f'Smoothed flux {key} {SELECT_BAND}  band')
ax2.set_xlabel('RA')
ax2.set_ylabel('')
ax2.grid(color="blue",linestyle=":")
ax2.contour(smoothed_image, levels=NCONT, colors='white', linewidths=1)
add_EastNorthFrame(40,155,ax2)


ax3 = fig.add_subplot(2, 2, 3)
im3 = ax3.imshow(data, origin='lower', cmap='jet', norm=norm)
ax3.set_title(f'Flux {key} {SELECT_BAND} band')
ax3.set_xlabel('X pix')
ax3.set_ylabel('Y pix')
ax3.grid(color="blue",linestyle=":")
ax3.contour(data, levels=NCONT, colors='white', linewidths=1)
add_EastNorthFrame(40,155,ax3)



ax4 = fig.add_subplot(2, 2, 4)
im4 = ax4.imshow(smoothed_image, origin='lower', cmap='jet', norm=norm)
#im4 = ax4.imshow(smoothed_image, origin='lower', cmap='jet')
ax4.set_title(f'Smoothed flux {key} {SELECT_BAND}  band')
ax4.set_xlabel('X pix')
ax4.grid(color="blue",linestyle=":")
ax4.contour(smoothed_image, levels=NCONT, colors='white', linewidths=1)
add_EastNorthFrame(40,155,ax4)


# Mise en page
plt.tight_layout()
plt.show()


In [ ]:
# List of maxima
df_maxima

In [ ]:
assert False

#### Most interesting maxima
- central maximum at (x,y) = (93, 105)
- One local maximum at (x,y) = (114,46)
- One local maximum at (x,y) = (146,126)

In [ ]:
ra_at_xy_centralmax, dec_at_xy_centralmax = wcs.wcs_pix2world(93, 105, 0)  # le 3e argument = 0 → origine '0-based' comme Python
delta_ra_at_xy_centralmax = (ra_at_xy_centralmax - target_ra)*3600
delta_dec_at_xy_centralmax = (dec_at_xy_centralmax - target_dec)*3600
print(f"relative position of centralmax x=93 y=105 is delta_ra = {delta_ra_at_xy_centralmax:.3f} arcsec, delta_dec = {delta_dec_at_xy_centralmax:.3f} arcsec ")

In [ ]:
ra_at_xy_s1, dec_at_xy_s1 = wcs.wcs_pix2world(114, 46, 0)  # le 3e argument = 0 → origine '0-based' comme Python
delta_ra_at_xy_s1 = (ra_at_xy_s1 - target_ra)*3600
delta_dec_at_xy_s1 = (dec_at_xy_s1 - target_dec)*3600
print(f"relative position of source1  x=114 y=46 is delta_ra = {delta_ra_at_xy_s1:.3f} arcsec, delta_dec = {delta_dec_at_xy_s1:.3f} arcsec ")

In [ ]:
ra_at_xy_s2, dec_at_xy_s2 = wcs.wcs_pix2world(146, 126, 0)  # le 3e argument = 0 → origine '0-based' comme Python
delta_ra_at_xy_s2 = (ra_at_xy_s2 - target_ra)*3600
delta_dec_at_xy_s2 = (dec_at_xy_s2 - target_dec)*3600
print(f"relative position of source2 x=146 y=126 is delta_ra = {delta_ra_at_xy_s1:.3f} arcsec, delta_dec = {delta_dec_at_xy_s1:.3f} arcsec ")

#### loop on images of sources
- Extract coordinates in ra,dec of images of the source

In [ ]:
# Nsource_images = 2 or 4
Nsource_images = 2
all_delta_ra_at_xy_s = np.zeros(Nsource_images)
all_delta_dec_at_xy_s = np.zeros(Nsource_images)
for idx in range(Nsource_images):
    xpix = df_maxima.iloc[idx+1]['x']
    ypix = df_maxima.iloc[idx+1]['y']
    ra_at_xy_s, dec_at_xy_s = wcs.wcs_pix2world(xpix, ypix, 0) 

    # take relative distance of the image wrt the lens in arcsec
    delta_ra_at_xy_s = (ra_at_xy_s - target_ra)*3600
    delta_dec_at_xy_s = (dec_at_xy_s - target_dec)*3600

    all_delta_ra_at_xy_s[idx] = delta_ra_at_xy_s
    all_delta_dec_at_xy_s[idx] = delta_dec_at_xy_s

## Header

### Find total exposure time

In [ ]:
#header

In [ ]:
exp_time = header["EXPTIME"]
print(f"Total exposure time of the image {exp_time} seconds")

## Plot sigma

In [ ]:
X = data.flatten()
Y = (1./np.sqrt(data_wt)).flatten()
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10, 4))
ax1.scatter(X, Y,marker="o",color="b",alpha=0.1)
ax1.set_yscale("log")
ax1.set_xscale("log")
ax1.set_xlim(0.00001,1)
ax1.set_ylim(0.028,0.06)
title = f'{key} {SELECT_BAND} band $\sigma_F$ vs Flux'
ax1.set_title(title)
ax1.set_xlabel("Flux")
ax1.set_ylabel("$\sigma_{Flux}$")

ax2.hist(Y,bins=50,histtype="step",color="b",lw=3)
ax2.set_title("standard deviation")
ax2.set_xlabel("$\sigma_{Flux}$")
cut_sigma = 0.042
ax2.axvline(cut_sigma,color="k",ls=":")

### In high sigma tail¶

In [ ]:
pixel_select = Y > cut_sigma

In [ ]:
X1 = X[pixel_select]
Y1 = Y[pixel_select]
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10, 4))
ax1.scatter(X1, Y1,marker="o",color="b",alpha=0.1)
ax1.set_yscale("log")
ax1.set_xscale("log")
ax1.set_xlim(0.00001,1)
#ax1.set_ylim(cut_sigma,0.06)
title = f'{key} {SELECT_BAND} band $\sigma_F$ vs Flux'
ax1.set_title(title)
ax1.set_xlabel("Flux")
ax1.set_ylabel("$\sigma_{Flux}$")

ax2.hist(Y1,bins=50,histtype="step",color="b",lw=3)
ax2.set_title("standard deviation")
ax2.set_xlabel("$\sigma_{Flux}$")

In [ ]:
F = data
sig = (1./np.sqrt(data_wt))

In [ ]:
Fsel = np.where(sig>cut_sigma, F,0.0)

In [ ]:
fig = plt.figure(figsize=(12, 6))

ax1 = fig.add_subplot(1, 2, 1, projection=wcs)
im1 = ax1.imshow(data, origin='lower', cmap='gray', norm=norm)
ax1.set_title(f'Flux {key} {SELECT_BAND} band')
ax1.set_xlabel('RA')
ax1.set_ylabel('Dec')
ax1.grid(color="blue",linestyle=":")
add_EastNorthFrame(40,155,ax1)

ax2 = fig.add_subplot(1, 2, 2, projection=wcs_wt)
im2 = ax2.imshow(Fsel, origin='lower', cmap='gray', norm=norm)
ax2.set_title(f'Flux sel {key} {SELECT_BAND}  band')
ax2.set_xlabel('RA')
ax2.set_ylabel('')
add_EastNorthFrame(40,155,ax2)

# Mise en page
plt.tight_layout()
plt.show()

### Remove high sigma tail

In [ ]:
pixel_select = Y < cut_sigma

In [ ]:
X1 = X[pixel_select]
Y1 = Y[pixel_select]
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10, 4))
ax1.scatter(X1, Y1,marker="o",color="b",alpha=0.1)
ax1.set_yscale("log")
ax1.set_xscale("log")
ax1.set_xlim(0.00001,1)
#ax1.set_ylim(cut_sigma,0.06)
title = f'{key} {SELECT_BAND} band $\sigma_F$ vs Flux'
ax1.set_title(title)
ax1.set_xlabel("Flux")
ax1.set_ylabel("$\sigma_{Flux}$")

ax2.hist(Y1,bins=50,histtype="step",color="b",lw=3)
ax2.set_title("standard deviation")
ax2.set_xlabel("$\sigma_{Flux}$")

### Check removed high sigma tail

In [ ]:
F = data
sig = (1./np.sqrt(data_wt))

In [ ]:
Fsel = np.where(sig<cut_sigma, F,0.0)

In [ ]:
fig = plt.figure(figsize=(12, 6))

ax1 = fig.add_subplot(1, 2, 1, projection=wcs)
im1 = ax1.imshow(data, origin='lower', cmap='gray', norm=norm)
ax1.set_title(f'Flux {key} {SELECT_BAND} band')
ax1.set_xlabel('RA')
ax1.set_ylabel('Dec')
ax1.grid(color="blue",linestyle=":")
add_EastNorthFrame(40,155,ax1)


ax2 = fig.add_subplot(1, 2, 2, projection=wcs_wt)
im2 = ax2.imshow(Fsel, origin='lower', cmap='gray', norm=norm)
ax2.set_title(f'Flux sel {key} {SELECT_BAND}  band')
ax2.set_xlabel('RA')
ax2.set_ylabel('')
ax2.grid(color="blue",linestyle=":")
add_EastNorthFrame(40,155,ax2)


# Mise en page
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 6))

ax1 = fig.add_subplot(1, 2, 1)
im1 = ax1.imshow(data, origin='lower', cmap='gray', norm=norm)
ax1.set_title(f'Flux {key} {SELECT_BAND} band')
ax1.set_xlabel('pixels (along RA)')
ax1.set_ylabel('picels (along Dec)')
ax1.grid(color="blue",linestyle=":")
add_EastNorthFrame(40,155,ax1)


ax2 = fig.add_subplot(1, 2, 2)
im2 = ax2.imshow(Fsel, origin='lower', cmap='gray', norm=norm)
ax2.set_title(f'Flux sel {key} {SELECT_BAND}  band')
ax2.set_xlabel('pixels (along RA)')
ax2.set_ylabel('')
ax2.grid(color="blue",linestyle=":")
add_EastNorthFrame(40,155,ax2)


# Mise en page
plt.tight_layout()
plt.show()

- I guess there is one source at pixels = (150,125) and one source at pixels (120,40)

In [ ]:
#ra_at_xy_s1, dec_at_xy_s1 = wcs.wcs_pix2world(150, 125, 0)  # le 3e argument = 0 → origine '0-based' comme Python
#ra_at_xy_s2, dec_at_xy_s2 = wcs.wcs_pix2world(120, 40, 0)  # le 3e argument = 0 → origine '0-based' comme Python

In [ ]:
#delta_ra_at_xy_s1 = (ra_at_xy_s1 - target_ra)*3600
#delta_dec_at_xy_s1 = (dec_at_xy_s1 - target_dec)*3600
#print(f"relative position of S1 is delta_ra = {delta_ra_at_xy_s1:.3f} arcsec, delta_dec = {delta_dec_at_xy_s1:.3f} arcsec ")

In [ ]:
#delta_ra_at_xy_s2 = (ra_at_xy_s2 - target_ra)*3600
#delta_dec_at_xy_s2 = (dec_at_xy_s2 - target_dec)*3600
#print(f"relative position of S2 is delta_ra = {delta_ra_at_xy_s2:.3f} arcsec, delta_dec = {delta_dec_at_xy_s2:.3f} arcsec ")

#### Remark about coordinate (ra,dec)
- RA increase toward left
- Dec increase toward up

### LensModel module

 $\texttt{LensModel}$ module
$\texttt{LensModel}$ and its sub-packages execute all the purely lensing related tasks of *lenstronomy*. This includes ray-shooting, solving the lens equation, arrival time computation and non-linear solvers to optimize lens models for specific image configurations. The module allows consistent integration with single and multi plane lensing and an arbitrary superpositions of lens models. There is a wide range of lens models available. For details we refer to to the online-documentation.

### Single plane lensing
As an example of initializing a single plane lens model as a super-position of an elliptical power-law potential and a singular isothermal sphere mass distribution and execute some basic routines:

In [ ]:
import lenstronomy

In [ ]:
lenstronomy.__version__

In [ ]:
# import the LensModel class #
from lenstronomy.LensModel.lens_model import LensModel

# specify the choice of lens models #
lens_model_list = ['EPL', 'SHEAR']

# setup lens model class with the list of lens models #
lensModel = LensModel(lens_model_list=lens_model_list)

# define parameter values of lens models #
#kwargs_spep = {'theta_E': 1.1, 'e1': 0.1, 'e2': 0.1, 'gamma': 2., 'center_x': 0.0, 'center_y': 0}
kwargs_spep = {'theta_E': 1.1, 'e1': 0.1, 'e2': 0.2, 'gamma': 2., 'center_x': 0.0, 'center_y': 0}
kwargs_shear = {'gamma1': -0.01, 'gamma2': .03}
kwargs_lens = [kwargs_spep, kwargs_shear]


# image plane coordinate #
# Need to guess where is one image
# theta_ra, theta_dec = target_ra, target_dec
#theta_ra, theta_dec = -1.3, .0
# Note this does not work later if I choose s2 !
theta_ra, theta_dec  = delta_ra_at_xy_s1, delta_dec_at_xy_s1

# source plane coordinate #
beta_ra, beta_dec = lensModel.ray_shooting(theta_ra, theta_dec, kwargs_lens)
# Fermat potential #
fermat_pot = lensModel.fermat_potential(x_image=theta_ra, y_image=theta_dec, x_source=beta_ra, y_source=beta_dec, kwargs_lens=kwargs_lens)

# Magnification #
mag = lensModel.magnification(theta_ra, theta_dec, kwargs_lens)

In [ ]:
print(f"angle-beta : ra = {beta_ra:.3f} arcsec, dec = {beta_dec:.3f} arcsec")

- According the initial guess where is the image, one can find the infered true source position and where are the other sources. 

In [ ]:
from lenstronomy.Plots import lens_plot
f, axex = plt.subplots(1, 1, figsize=(5, 5), sharex=False, sharey=False)
lens_plot.lens_model_plot(axex, lensModel=lensModel, kwargs_lens=kwargs_lens, sourcePos_x=beta_ra, sourcePos_y=beta_dec, point_source=True, with_caustics=True, fast_caustic=True, coord_inverse=True)
#lens_plot.lens_model_plot(axex[1], lensModel=lensModel_mp, kwargs_lens=kwargs_lens_mp, sourcePos_x=beta_ra, sourcePos_y=beta_dec, point_source=True, with_caustics=True, fast_caustic=True, coord_inverse=False)
axex.grid()
f.show()

In [ ]:
# keep the imports and variables from above #
# import the lens equation solver class #
from lenstronomy.LensModel.Solver.lens_equation_solver import LensEquationSolver

# specifiy the lens model class to deal with #
solver = LensEquationSolver(lensModel)

# solve for image positions provided a lens model and the source position #
theta_ra, theta_dec = solver.image_position_from_source(beta_ra, beta_dec, kwargs_lens)

# the magnification of the point source images #
mag = lensModel.magnification(theta_ra, theta_dec, kwargs_lens)

- Then the solver create the vectors of images position (theta_ra,theta_dec)

In [ ]:
print(f"theta_ra = {theta_ra} (arcsec), theta_dec = {theta_dec} (arcsec)")

#### $\texttt{LightModel}$ module

The $\texttt{LightModel}$ class provides the functionality to describe galaxy surface brightnesses. $\texttt{LightModel}$ supports various analytic profiles as well as representations in shapelet basis sets. Any superposition of different profiles is allowed.

The parameter levelig the amplitude of the surface brightness of a certain profile is named 'amp'. The units are not further specified and are effectively reflecting a surface brightness quantitiy integrated over the unit of angle square. In the $\texttt{SimulationAPI}$ module, the user can conveniently chose astronomical magnitudes as inputs to the profiles, provided the magnitude zero point is declared. Have a look at the specific notebook to see a demonstration.

In [ ]:
# import the LightModel class #
from lenstronomy.LightModel.light_model import LightModel
# set up the list of light models to be used #
source_light_model_list = ['SERSIC']
lightModel_source = LightModel(light_model_list=source_light_model_list)
lens_light_model_list = ['SERSIC_ELLIPSE']
lightModel_lens = LightModel(light_model_list=lens_light_model_list)
# define the parameters #
kwargs_light_source = [{'amp': 100, 'R_sersic': 0.1, 'n_sersic': 1.5, 'center_x': beta_ra, 'center_y': beta_dec}]
import lenstronomy.Util.param_util as param_util
e1, e2 = param_util.phi_q2_ellipticity(phi=0.5, q=0.7)
kwargs_light_lens = [{'amp': 1000, 'R_sersic': 0.1, 'n_sersic': 2.5, 'e1': e1, 'e2': e2, 'center_x': 0.1, 'center_y': 0}]

# evaluate surface brightness at a specific position #
flux = lightModel_lens.surface_brightness(x=1, y=1, kwargs_list=kwargs_light_lens)

### $\texttt{PointSource}$ module
To accurately predict and model the position and flux of point sources, one has to apply different numerical procedures than for extended surface brightness features. The $\texttt{PointSource}$ class manages the different options in describing point sources (e.g. in the image plane or source plane, with fixed magnification or allowed with individual variations thereof) and provides a homogeneous interface to access image positions and magnifications.

In [ ]:
# import the PointSource class #
from lenstronomy.PointSource.point_source import PointSource

# unlensed source position #
point_source_model_list = ['SOURCE_POSITION']
pointSource = PointSource(point_source_type_list=point_source_model_list, lens_model=lensModel, fixed_magnification_list=[True])
kwargs_ps = [{'ra_source': beta_ra, 'dec_source': beta_dec, 'source_amp': 100}]
# return image positions and amplitudes #
x_pos, y_pos = pointSource.image_position(kwargs_ps=kwargs_ps, kwargs_lens=kwargs_lens)
point_amp = pointSource.image_amplitude(kwargs_ps=kwargs_ps, kwargs_lens=kwargs_lens)

# lensed image positions (solution of the lens equation) #
point_source_model_list = ['LENSED_POSITION']
pointSource = PointSource(point_source_type_list=point_source_model_list, lens_model=lensModel, fixed_magnification_list=[False])
kwargs_ps = [{'ra_image': theta_ra, 'dec_image': theta_dec, 'point_amp': np.abs(mag)*30}]
# return image positions and amplitudes #
x_pos, y_pos = pointSource.image_position(kwargs_ps=kwargs_ps, kwargs_lens=kwargs_lens)
point_amp = pointSource.image_amplitude(kwargs_ps=kwargs_ps, kwargs_lens=kwargs_lens)

### Make the Data module

In [ ]:
noise_map = 1./np.sqrt(data_wt)

#### Make the PSF

La caméra ACS/WFC (Wide Field Channel) du Hubble Space Telescope a une échelle angulaire (pixel scale) de :

- ACS/WFC : ~0.0495 arcsec/pixel
- ACS/HRC (High Resolution Channel, maintenant inactive) : ~0.028 arcsec/pixel

In [ ]:
from lenstronomy.Data.psf import PSF

# 1.kwargs pour la PSF
kwargs_psf = {
    'psf_type': 'GAUSSIAN',
    'fwhm': 0.11,
    'pixel_size': pixel_scale
}

In [ ]:
# 2. Définir la PSF comme gaussienne approximative

#psf_class = PSF(psf_type='PIXEL', kernel_point_source=psf_kernel)
# FWHM typique en z-band (ACS) : 0.11 arcsec
fwhm = 0.11  # arcsec
psf_class = PSF(psf_type='GAUSSIAN', fwhm=fwhm, pixel_size=pixel_scale)

In [ ]:
kernel = psf_class.kernel_point_source

#### Make the data (True observations)

In [ ]:
from lenstronomy.Data.imaging_data import ImageData

In [ ]:
# --- Exemple ---
kwargs_data = {
    'image_data': data,
    'noise_map': noise_map,
    'exposure_time': None,
    'background_rms': None,
    'exposure_time': exp_time,
    'ra_at_xy_0': delta_ra_at_xy_0,  
    'dec_at_xy_0': delta_dec_at_xy_0,
    'transform_pix2angle': transform_pix2angle 
}

In [ ]:
# 3. Définir l'objet ImageData

# --- Créer l'objet ImageData ---
data_class = ImageData(**kwargs_data)

### Check the Forward model of the predicted image

#### Pixel grid

In [ ]:
pixel_scale

In [ ]:
# import the PixelGrid() class #
from lenstronomy.Data.pixel_grid import PixelGrid
deltaPix = pixel_scale
transform_pix2anglepixgrid = np.array([[-1, 0], [0, 1]]) * deltaPix  # linear translation matrix of a shift in pixel in a shift in coordinates
kwargs_pixel = {'nx': 200, 'ny': 200,  # number of pixels per axis
                'ra_at_xy_0': delta_ra_at_xy_0,  # RA at pixel (0,0)
                'dec_at_xy_0': delta_dec_at_xy_0,  # DEC at pixel (0,0)
                'transform_pix2angle': transform_pix2anglepixgrid} 
pixel_grid = PixelGrid(**kwargs_pixel)
# return the list of pixel coordinates #
x_coords, y_coords = pixel_grid.pixel_coordinates
# compute pixel value of a coordinate position #
x_pos, y_pos = pixel_grid.map_coord2pix(ra=0, dec=0)
# compute the coordinate value of a pixel position #
ra_pos, dec_pos = pixel_grid.map_pix2coord(x=20, y=10)

### $\texttt{ImSim}$ module
The $\texttt{ImSim}$ module simulates images. At its core is the $\texttt{ImageModel}$ class. It is the interface to combine all the different components, $\texttt{LensModel}$, $\texttt{LightModel}$, $\texttt{PointSource}$ and $\texttt{Data}$ to model images. The $\texttt{LightModel}$ can be used to model lens light (unlensed) or source light (lensed). $\texttt{ImSim}$ is fully supporting all functionalities in each component. Additionaly, numerical precision arguments can be set in how to model the image.

#### Simulate image
We simulate an image with an instance of $\texttt{ImageModel}$ that got instances of the classes we created above. We can define two different $\texttt{LightModel}$ instances for the lens and source light. Additionally, we can define the sub-pixel ray-tracing resolution and whether the PSF convolution is applied on the higher resolution ray-tracing image or on the degraded pixel image.

In this example, we do not simulate point source. You can look at the notebooks dedicated to model quads and doubles and also for time-delya cosmography.

In [ ]:
# import the ImageModel class #
from lenstronomy.ImSim.image_model import ImageModel
# define the numerics #
kwargs_numerics = {'supersampling_factor': 1, # each pixel gets super-sampled (in each axis direction) 
                  'supersampling_convolution': False}

# initialize the Image model class by combining the modules we created above #
imageModel = ImageModel(data_class=pixel_grid, psf_class=psf_class, lens_model_class=lensModel,
                        source_model_class=lightModel_source,
                        lens_light_model_class=lightModel_lens,
                        point_source_class=None, # in this example, we do not simulate point source.
                        kwargs_numerics=kwargs_numerics)
# simulate image with the parameters we have defined above #
image = imageModel.image(kwargs_lens=kwargs_lens, kwargs_source=kwargs_light_source,
                         kwargs_lens_light=kwargs_light_lens, kwargs_ps=kwargs_ps)

# we can also add noise #
import lenstronomy.Util.image_util as image_util
#exp_time = 100  # exposure time to quantify the Poisson noise level
background_rms = 0.001  # background rms value
poisson = image_util.add_poisson(image, exp_time=exp_time)
bkg = image_util.add_background(image, sigma_bkd=background_rms)
image_noisy = image + bkg + poisson

f, axes = plt.subplots(1, 2, figsize=(8, 4), sharex=False, sharey=False)
axes[0].matshow(np.log10(image), origin='lower')
axes[1].matshow(np.log10(image_noisy), origin='lower')

add_EastNorthFrame(40,155,axes[0])
add_EastNorthFrame(40,155,axes[1])


f.tight_layout()
plt.show()

#### Linear inversion
Parameter corresponding to an an amplitude of a surface brightness distribution have a linear response on the predicted flux values in the data and can be infered by a linear inversion. This can reduce the number of non-linear parameters fastly, depending on the source complexity to be modelled.

The $\texttt{ImageLinearFit}$ class performs this computation. The class inherits the $\texttt{ImageModel}$ class but instead of the $\texttt{PixelGrid}$ instance, it requires an instance of the full $\texttt{ImageData}$ class including the data (which we use the mock image created above) and the noise properties therof.

In the example of this notebook, we have 6 linear parameters, the 4 point source amplitudes and the amplitudes of the Sersic profile of the lens and source. *lenstronomy* automatically identifies those parameters and can recover those values (data permitting).

In [ ]:
from lenstronomy.Data.imaging_data import ImageData
# update the data with the noisy image and its noise properties #
kwargs_data = {'image_data': image_noisy,
               'background_rms': background_rms,
               'exposure_time': exp_time,
               'ra_at_xy_0': delta_ra_at_xy_0,  
               'dec_at_xy_0': delta_dec_at_xy_0,
               'transform_pix2angle': transform_pix2angle} 

data_class = ImageData(**kwargs_data)

from lenstronomy.ImSim.image_linear_solve import ImageLinearFit
image_linear_fit = ImageLinearFit(data_class=data_class, psf_class=psf_class, lens_model_class=lensModel,
                        source_model_class=lightModel_source,
                        lens_light_model_class=lightModel_lens,
                        point_source_class=pointSource, 
                        kwargs_numerics=kwargs_numerics)

# we do not require the knowledge of the linear parameters #
del kwargs_light_source[0]['amp']
# apply the linear inversion to fit for the noisy image #
image_reconstructed, _, _, _ = image_linear_fit.image_linear_solve(kwargs_lens=kwargs_lens, kwargs_source=kwargs_light_source, 
                              kwargs_lens_light=kwargs_light_lens, kwargs_ps=kwargs_ps)

In [ ]:
# illustrate fit #
from lenstronomy.Plots.model_plot import ModelPlot
kwargs_model = {'lens_model_list': lens_model_list, 'source_light_model_list': source_light_model_list,
               'lens_light_model_list': lens_light_model_list} #, 'point_source_model_list': point_source_model_list}
kwargs_params = {'kwargs_lens': kwargs_lens, 'kwargs_source': kwargs_light_source,
                 'kwargs_lens_light': kwargs_light_lens, 'kwargs_ps': kwargs_ps}
lensPlot = ModelPlot([[kwargs_data, kwargs_psf, kwargs_numerics]], kwargs_model, kwargs_params, arrow_size=0.02)

f, axes = plt.subplots(1, 2, figsize=(8, 4), sharex=False, sharey=False)
lensPlot.data_plot(ax=axes[0])
lensPlot.model_plot(ax=axes[1])
f.tight_layout()
plt.show()

### $\texttt{Sampling}$ Module
The $\texttt{Sampling}$ Module consists of three major classes: 

- The $\texttt{Likelihood}$ class manages the specific likelihood function, consisting of the imaging likelihood and potential other data and constraints.
- The $\texttt{Param}$ class handles all the model choices and the parameters going in it. These classes handle all the model choices of the user and mitigate them to the external modules and from the external modules back to *lenstronomy*.
- The $\texttt{Sampler}$ class gives specific examples how the $\texttt{Likelihood}$ and $\texttt{Param}$ classes can be used to execute specific samplers. This last class can also be replaced by other sampling methods by the user.


#### $\texttt{Param}$ class
The $\texttt{Param}$ class is the interface of the *lenstronomy* conventions of parameters (list of keyword arguments) and the fitting conventions of $\texttt{CosmoHammer}$ or $\texttt{emcee}$ (single array consisting all parameters). The $\texttt{Param}$ class enables the user to set further options:

- keep certain parameters fixed
- handling of the linear parameters
- provide additional constraints on the modelling (e.g. fix source profile to point source position etc)

With the $\texttt{Param}$ class, lenstronomy can be integrated robustly with other sampling methods.




<div class="alert alert-warning">

**ATTENTION:** lenstronomy, by default, solves for the amplitude values of surface brighthness functions (in the lens and source) specified in the model by 'amp' through a linear minimization at each likelihood evaluation step. Solving a linear equation is more efficient than sampling the parameters in the non-linear exploration. Hence, the output of a sampling chain (be it PSO, MCMC or any other sampler used) do not contain the linear amplitudes and are set as =0.
To retriev the linear amplitude parameters, you have to re-do the linear inversion in post processing. This can be done with the $\texttt{LensModelPlot}$ class (see block further down) or with the $\texttt{ImageReconstruction}$ class in the $\texttt{Analysis}$ sub-package.

</div>

In [ ]:
# import the parameter handling class #
from lenstronomy.Sampling.parameters import Param
# set options for constraint parameters #
# Here we join the position of the lens light and mass center. 
# There are more options available in the Param() class.
kwargs_constraints = {'joint_lens_with_light': [[0, 0, ['center_x', 'center_y']]]}

# list fixed parameters #
kwargs_fixed_lens = [{'gamma': 2.}, {'ra_0': 0, 'dec_0': 0}, {}]
kwargs_fixed_source = [{}]
kwargs_fixed_lens_light = [{}]

# optional: list lower and upper bounds in parameter space, otherwise it will be using lenstronomy default values! #
# list lower bounds in parameter space #
kwargs_lower_lens = [{'theta_E': 0, 'e1': -0.5, 'e2': -0.5, 'gamma': 1.5, 'center_x': -10., 'center_y': -10},
                     {'gamma1': -0.5, 'gamma2': -0.5}]
kwargs_lower_source = [{'R_sersic': 0.001, 'n_sersic': .5, 'center_x': -10, 'center_y': -10}]
kwargs_lower_lens_light = [{'R_sersic': 0.001, 'n_sersic': 0.5, 'e1': -0.5, 'e2': -0.5, 'center_x': -10, 'center_y': -10}]
kwargs_lower_ps = None

# list upper bounds in parameter space #
kwargs_upper_lens = [{'theta_E': 10, 'e1': 0.5, 'e2': 0.5, 'gamma': 2.5, 'center_x': 10., 'center_y': 10},
                     {'gamma1': 0.5, 'gamma2': 0.5}]
kwargs_upper_source = [{'R_sersic': 10, 'n_sersic': 5., 'center_x': 10, 'center_y': 10}]
kwargs_upper_lens_light = [{'R_sersic': 10, 'n_sersic': 5., 'e1': 0.5, 'e2': 0.5, 'center_x': 10, 'center_y': 10}]
kwargs_upper_ps = None

# make instance of parameter class with given model options, constraints and fixed parameters #
param = Param(kwargs_model, kwargs_fixed_lens, kwargs_fixed_source,
              kwargs_fixed_lens_light, kwargs_fixed_ps=None, 
              kwargs_lower_lens=kwargs_lower_lens, kwargs_lower_source=kwargs_lower_source,
              kwargs_lower_lens_light=kwargs_lower_lens_light, kwargs_lower_ps=kwargs_lower_ps,
              kwargs_upper_lens=kwargs_upper_lens, kwargs_upper_source=kwargs_upper_source,
              kwargs_upper_lens_light=kwargs_upper_lens_light, kwargs_upper_ps=kwargs_upper_ps,
              kwargs_lens_init=kwargs_lens, **kwargs_constraints)

# the number of non-linear parameters and their names #
num_param, param_list = param.num_param()

# parameter array for fitting routine #
param_array = param.kwargs2args(kwargs_lens, kwargs_light_source, kwargs_light_lens, kwargs_ps)

# recover keyword arguments list for lenstronomy from a parameter array of the fitting routine #
kwargs_out = param.args2kwargs(param_array)

# print settings #
param.print_setting()

#### $\texttt{Likelihood}$ class

The $\texttt{Likelihood}$ class performs the likelihood computation. It combines the $\texttt{ImSim}$ module with the $\texttt{Param}$ module. The $\texttt{Likelihood}$ class can evaluate imaging likelihoods, image position and time-delay likelihood with various options. For details we refer to the class documentation.

In [ ]:
kwargs_likelihood = {'source_marg': False,
                    'source_position_tolerance': 0.001,}
single_band = [[kwargs_data, kwargs_psf, kwargs_numerics]]
kwargs_data_joint = {'multi_band_list': single_band, 'multi_band_type': 'multi-linear'}

from lenstronomy.Sampling.likelihood import LikelihoodModule
likelihoodModule = LikelihoodModule(kwargs_data_joint, kwargs_model, param_class=param, **kwargs_likelihood)

logL = likelihoodModule.logL(args=param_array)

#### $\texttt{Sampler}$ class

The $\texttt{Sampler}$ class performs the sampling of the likelihood. The class requires an instance of the $\texttt{Likelihood}$ class. This class is also meant to showcase the sampling of the *lenstronomy* likelihood. We emphasise that the specific parameter space and complexity may warry significantly and depending on the problem, different approaches can be more efficient.

<div class="alert alert-warning">

**Warning:** This particular example was run with a very small and insufficiently long Particle Swarm Optimization. This is for illustrative purpose only. You need to change the 'n_particles' and 'n_iterations' settings and make sure the PSO is sufficiently converged to draw conclusions from the best fit.

</div>

In [ ]:
# initial guess of non-linear parameters, we chose different starting parameters than the truth #
kwargs_lens_init = [{'theta_E': 1.2, 'e1': 0, 'e2': 0, 'gamma': 2., 'center_x': 0., 'center_y': 0},
                    {'gamma1': 0, 'gamma2': 0}]
kwargs_source_init = [{'R_sersic': 0.03, 'n_sersic': 1., 'center_x': 0, 'center_y': 0}]
kwargs_lens_light_init = [{'R_sersic': 0.1, 'n_sersic': 1, 'e1': 0, 'e2': 0, 'center_x': 0, 'center_y': 0}]

# initial spread in parameter estimation #
kwargs_lens_sigma = [{'theta_E': 0.3, 'e1': 0.5, 'e2': 0.5, 'gamma': .2, 'center_x': 0.1, 'center_y': 0.1},
                     {'gamma1': 0.05, 'gamma2': 0.05}]
kwargs_source_sigma = [{'R_sersic': 0.1, 'n_sersic': .5, 'center_x': .1, 'center_y': 0.1}]
kwargs_lens_light_sigma = [{'R_sersic': 0.1, 'n_sersic': 0.2, 'e1': 0.2, 'e2': 0.2, 'center_x': .1, 'center_y': 0.1}]

param_init = param.kwargs2args(kwargs_lens_init, kwargs_source_init, kwargs_lens_light_init, kwargs_ps=None)
param_sigma = param.kwargs2args(kwargs_lens_sigma, kwargs_source_sigma, kwargs_lens_light_sigma, kwargs_ps=None)

upper_start = np.array(param_init) + np.array(param_sigma)
lower_start = np.array(param_init) - np.array(param_sigma)

from lenstronomy.Sampling.sampler import Sampler
sampler = Sampler(likelihoodModule=likelihoodModule)

result, chain_properties = sampler.pso(n_particles=200, n_iterations=10, lower_start=lower_start, upper_start=upper_start)

kwargs_result = param.args2kwargs(result)


#### $\texttt{FittingSequence}$ class

The $\texttt{FittingSequence}$ class allows to perform a PSO and/or MCMC run. The user can run a sequence of fitting routines, applied one after the other with taking the results of the previous routine as an input of the next one. The user can specify (optionaly) to keep one or multiple parameter classes (lens model, source model, lens light model and source model) fixed during the fitting process or parameter inference.

The user further needs to specify:
- define input guess and uncertainties and uncertainties
- hard bounds on parameters
- number of particles and number of iterations being performed by the PSO/MCMC
- enables MPI run through a set of fitting sequences

$\texttt{FittingSequence}$ does not rely on instances of other *lenstronomy* classes. It can be constructed from elements in the form of keyword arguments and lists thereof in the *lenstronomy* conventions. This enables a save and reliable execution of tasks within $\texttt{FittingSequence}$ on non-local platforms, such as high performance clusters.

MPI mode can be set in the initialization of the $\texttt{FittingSequence}$ class. Multi-threading can be set (if available) for each individual sequence in their keyword arguments with 'threadCount'.

### $\texttt{Workflow}$ Module
The $\texttt{Workflow}$ Module manages the execution of non-linear fitter (in our case the PSO) and the parameter inference (in our case the CosmoHammer emcee). The two main classes are $\texttt{Param}$ and $\texttt{FittingSequence}$. These classes handle all the model choices of the user and mitigate them to the external modules and from the external modules back to *lenstronomy*.


In [ ]:
# for this part of the code, the emcee MCMC sampler needs to be installed
try:
    import emcee
except:
    !pip install emcee

lens_params = [kwargs_lens_init, kwargs_lens_sigma, kwargs_fixed_lens, kwargs_lower_lens, kwargs_upper_lens]
source_params = [kwargs_source_init, kwargs_source_sigma, kwargs_fixed_source, kwargs_lower_source, kwargs_upper_source]
lens_light_params = [kwargs_light_lens, kwargs_lens_light_sigma, kwargs_fixed_lens_light, kwargs_lower_lens_light, kwargs_upper_lens_light]

kwargs_params = {'lens_model': lens_params,
                'source_model': source_params,
                'lens_light_model': lens_light_params}

band_list = [kwargs_data, kwargs_psf, kwargs_numerics]
multi_band_list = []
multi_band_list.append(band_list)


from lenstronomy.Workflow.fitting_sequence import FittingSequence
fitting_seq = FittingSequence(kwargs_data_joint, kwargs_model, kwargs_constraints, kwargs_likelihood, kwargs_params, mpi=False)

fitting_kwargs_list = [['PSO', {'sigma_scale': 1, 'n_particles': 200, 'n_iterations': 200, 'threadCount': 1}],
                       ['MCMC', {'n_burn': 200, 'n_run': 200, 'n_walkers': 100, 'sigma_scale': .01, 'threadCount': 1}]]

chain_list = fitting_seq.fit_sequence(fitting_kwargs_list)
kwargs_result = fitting_seq.best_fit()

<div class="alert alert-warning">

**Warning:** This particular example was run with a very small and insufficiently long MCMC chain. This is for illustrative purpose only. You need to change the 'n_burn' and 'n_run' settings and make sure the chains are sufficiently converged to draw conclusions from the posteriors.

</div>


#### $\texttt{LensModelPlot}$ class
The $\texttt{LensModelPlot}$ facilitates the plotting of models, comparison with the data etc. This class is also meant to illustrate the post-processing.

In [ ]:
modelPlot = ModelPlot([[kwargs_data, kwargs_psf, kwargs_numerics]], kwargs_model, kwargs_result, arrow_size=0.02, cmap_string="gist_heat")
    
f, axes = plt.subplots(2, 3, figsize=(16, 8), sharex=False, sharey=False)

modelPlot.data_plot(ax=axes[0,0])
modelPlot.model_plot(ax=axes[0,1])
modelPlot.normalized_residual_plot(ax=axes[0,2], v_min=-6, v_max=6)
modelPlot.source_plot(ax=axes[1, 0], deltaPix_source=0.01, numPix=100)
modelPlot.convergence_plot(ax=axes[1, 1], v_max=1)
modelPlot.magnification_plot(ax=axes[1, 2])
f.tight_layout()
f.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0., hspace=0.05)
plt.show()

f, axes = plt.subplots(2, 3, figsize=(16, 8), sharex=False, sharey=False)

modelPlot.decomposition_plot(ax=axes[0,0], text='Lens light', lens_light_add=True, unconvolved=True)
modelPlot.decomposition_plot(ax=axes[1,0], text='Lens light convolved', lens_light_add=True)
modelPlot.decomposition_plot(ax=axes[0,1], text='Source light', source_add=True, unconvolved=True)
modelPlot.decomposition_plot(ax=axes[1,1], text='Source light convolved', source_add=True)
modelPlot.decomposition_plot(ax=axes[0,2], text='All components', source_add=True, lens_light_add=True, unconvolved=True)
modelPlot.decomposition_plot(ax=axes[1,2], text='All components convolved', source_add=True, lens_light_add=True, point_source_add=True)
f.tight_layout()
f.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0., hspace=0.05)
plt.show()
print(kwargs_result)

In [ ]:
from lenstronomy.Plots import chain_plot

# here we get the truths of the input that we aim to recover in the MCMC chain with the help of the Param() class.
# we can also get the Param() class instance from the FittingSequence() instancd
param = fitting_seq.param_class
param_array_truths = param.kwargs2args(kwargs_lens, kwargs_light_source, kwargs_light_lens, kwargs_ps)

for i in range(len(chain_list)):
    chain_plot.plot_chain_list(chain_list, i)
    
sampler_type, samples_mcmc, param_mcmc, dist_mcmc  = chain_list[1]
print("number of non-linear parameters in the MCMC process: ", len(param_mcmc))
print("parameters in order: ", param_mcmc)
print("number of evaluations in the MCMC process: ", np.shape(samples_mcmc)[0])

import corner  # pip install corner  (if you have not installed it)

n, num_param = np.shape(samples_mcmc)
plot = corner.corner(samples_mcmc[:,:8], labels=param_mcmc[:8], show_titles=True, truths=param_array_truths[:8])
plot.show()
plot = corner.corner(samples_mcmc[:,8:], labels=param_mcmc[8:], show_titles=True, truths=param_array_truths[8:])
plot.show()

### Further ressources

This notebook ends here. There are almost certainly many aspects or features that you were looking for that were not presented above. For additional features of each class, we refer to the documentation of the specific classes.
Here we provide few other pointers to notebooks that might be useful of features beyond of what was presented here:

- [Units, coordiante system and parameter definitions in lenstronomy](https://github.com/sibirrer/lenstronomy_extensions/blob/main/lenstronomy_extensions/Notebooks/units_coordinates_parameters.ipynb)
- [SimulationAPI module for convenient simulations according to different instrument configurations](https://github.com/sibirrer/lenstronomy_extensions/blob/main/lenstronomy_extensions/Notebooks/simulation_api.ipynb)
- Modeling of point sources ([quad](https://github.com/sibirrer/lenstronomy_extensions/blob/main/lenstronomy_extensions/Notebooks/quad_model.ipynb), [double](https://github.com/sibirrer/lenstronomy_extensions/blob/main/lenstronomy_extensions/Notebooks/double_model.ipynb), [catalogue level](https://github.com/sibirrer/lenstronomy_extensions/blob/main/lenstronomy_extensions/Notebooks/catalogue%20modelling.ipynb))

And many more notebooks available on the [lenstronomy_extension](https://github.com/lenstronomy/lenstronomy-tutorials) repository!